# HW3 Q4 [10 pts]



## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>


All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  



<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`load_data()`

`exclude_no_pickuplocations()`

`exclude_no_tripdistance()`

`include_fare_range()`

`get_highest_tip()`

`get_total_toll()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from NYC Taxi Trips during auto-grading. You do not need to write code for unreasonable scenarios.  

Since the overall correctness of your code will require multiple function to work together correctly (i.e., all methods are interdepedent), implementing only a subset of the functions likely will lead to a low score.

### Helper functions

You are permitted to write additional helper functions, or use additional instance variables so long as the previously described functions work as required.  

#### Pyspark Imports
<span style="color:red">*Please don't modify the below cell*</span>

In [1]:
import pyspark
from pyspark.sql import SQLContext

#### Define Spark Context
<span style="color:red">*Please don't modify the below cell*</span>

In [2]:
sc
sqlContext = SQLContext(sc)

### Student Section - Please compete all the functions below

#### Function to return GT Username

In [3]:
def user():
        """
        :return: string
        your GTUsername, NOT your 9-Digit GTId  
        """         
        return 'jmickle6'

#### Function to load data

In [83]:
def load_data(gcp_storage_path):
    """
        :param gcp_storage_path: string (full gs path including file name e.g gs://bucket_name/data.csv) 
        :return: spark dataframe  
    """
    
    ################################################################
    df = spark.read.option("header",True) \
              .csv(gcp_storage_path)
        
    df = df.withColumn('trip_distance', df.trip_distance.cast('float')) \
        .withColumn('fare_amount', df.fare_amount.cast('float')) \
        .withColumn('tip_amount', df.tip_amount.cast('float')) \
        .withColumn('tolls_amount', df.tolls_amount.cast('float'))
    ################################################################
    
    
        
    return df

#### Function to exclude trips that don't have a pickup location

In [37]:
def exclude_no_pickuplocations(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    df = df.filter(df.PULocationID.isNotNull())
    ################################################################
    
    return df

#### Function to exclude trips with no distance

In [43]:
def exclude_no_tripdistance(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    df = df.filter((df.trip_distance.isNotNull()) & (df.trip_distance > 0))    
    ################################################################
    
    return df

#### Function to include fare amount between the range of 20 to 60 Dollars

In [45]:
def include_fare_range(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    df = df.filter((df.fare_amount >= 20) & (df.fare_amount <= 60))      
    ################################################################
    
    return df

#### Function to get the highest tip amount

In [58]:
def get_highest_tip(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: float (rounded to 2 digits)  
    """
    
    ################################################################
    max_tip = df.agg({"tip_amount":"max"}).collect()[0]["max(tip_amount)"]
    ################################################################
    
    return max_tip

#### Function to get total toll amount

In [70]:
def get_total_toll(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: float (rounded to 2 digits)  
    """
    
    ################################################################
    total_toll = df.withColumn('tolls_amount', df.tolls_amount.cast('integer')).agg({'tolls_amount':'sum'}).collect()[0]["sum(tolls_amount)"]
    ################################################################
    
    return total_toll

### Run above functions and print

#### Uncomment the cells below and test your implemented functions

#### Load data from yellow_tripdata_2019-01.csv 

In [84]:
gcp_storage_path = 'gs://jmickle6/yellow_tripdata_2019-01.csv'
df = load_data(gcp_storage_path)
df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



#### Print total numbers of rows in the dataframe

In [85]:
df.count()

7667792

#### Print total number of rows in the dataframe after excluding trips with no pickup location

In [86]:
df_no_pickup_locations = exclude_no_pickuplocations(df)
df_no_pickup_locations.count()

7667792

#### Print total number of rows in the dataframe after exclude trips with no distance

In [87]:
df_no_trip_distance = exclude_no_tripdistance(df_no_pickup_locations)
df_no_trip_distance.count()

7613022

#### Print total number of rows in the dataframe after including trips with fair amount between the range of 20 to 60 Dollars

In [88]:
df_include_fare_range = include_fare_range(df_no_trip_distance)
df_include_fare_range.count()

957105

#### Print the highest tip amount

In [89]:
max_tip = get_highest_tip(df_include_fare_range)
print(max_tip)

444.799987793


#### Print the total toll amount

In [90]:
total_toll = get_total_toll(df_include_fare_range)
print(total_toll)

1824596
